In [55]:
import dhlab as dh
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dhlab.api.dhlab_api as api


In [370]:
import ipydagred3 as dp

In [6]:
from random import random
import math
from IPython.display import Markdown

In [7]:
a = pd.Series([int(random()*10) for i in range(100)])
a.describe()

count    100.000000
mean       4.350000
std        2.815452
min        0.000000
25%        2.000000
50%        4.000000
75%        7.000000
max        9.000000
dtype: float64

In [8]:
hist = np.histogram(a)

In [45]:
def series_hist(a):
    hist = np.histogram(a)
    hst = pd.DataFrame(hist).transpose()
    hst.columns = ["counts","Position"]
    return hst

In [140]:
import altair as alt

def plot_series(a):
    return alt.Chart(series_hist(a)).mark_bar().encode(
        alt.X("Position", bin=False),
        y='counts',
    ).properties(
        width=300,
        height=100
    )

Theoretical mean and variance

In [10]:
N = 10
c = pd.Series(list(range(N))*10)

In [11]:
c.describe()

count    100.000000
mean       4.500000
std        2.886751
min        0.000000
25%        2.000000
50%        4.500000
75%        7.000000
max        9.000000
dtype: float64

$\sqrt{\frac{\sum{(x-L/2)^2}}{N-1}}$

In [479]:
corpus = dh.Corpus(doctype="digibok", from_year = 2000, subject='historie', limit = 20000).frame

In [558]:
corpus = pd.read_excel("corpus_covid.xlsx", index_col = 0)

In [559]:
#corpus.reset_index()

In [560]:
concs_left = api.word_concordance(dhlabid=list(corpus.reset_index().dhlabid), words=['doser'], before = 10, after=0, limit=5000)

In [561]:
concs_right = api.word_concordance(dhlabid=list(corpus.reset_index().dhlabid), words=['doser'], before = 0, after=10, limit = 5000)

In [562]:
left_lists = concs_left.before.fillna('').apply(lambda x: x.split())
right_lists = concs_right.after.fillna('').apply(lambda x: x.split())

In [563]:
left_words = {}
for l in left_lists:
    for x in l:
        if x in left_words:
            left_words[x].append(len(l) - 1 - l.index(x))
        else:
            left_words[x] = [len(l) - 1 - l.index(x)]

In [564]:
right_words = {}
for l in right_lists:
    for x in l:
        if x in right_words:
            right_words[x].append(l.index(x))
        else:
            right_words[x] = [l.index(x)]

In [565]:
left_set = {w:pd.Series(left_words[w]).describe()[['count','mean', 'std']] for w in left_words if len(left_words[w]) >= 5}

In [566]:
right_set = {w:pd.Series(right_words[w]).describe()[['count','mean', 'std']] for w in right_words if len(right_words[w]) >= 5}

In [567]:
left_df = pd.DataFrame(left_set).transpose()
left_df['score'] = left_df['mean']*(left_df['std']+0.3)
#left_df['CV'] = 100*left_df['std']/left_df['mean']

In [568]:
right_df = pd.DataFrame(right_set).transpose()
right_df['score'] = right_df['mean']*(right_df['std']+0.3)
#right_df['CV'] = 100*right_df['std']/right_df['mean']

In [569]:
cutoff = 20
(right_df[(right_df['count'] >= cutoff) & (right_df['mean']>4.3) & (right_df['mean'] < 4.7)].sort_values(by=['score'])
 .head(20).dropna()
 .style.format(precision=2)
  .set_table_styles([{'selector':"th", 'props':'font-size:18px'},
                   {'selector':'td', 'props':'font-size:18px'},
                   {'selector':'tc', 'props':'font-size:18px'}])
)

,count,mean,std,score
mellomrom,45.00,4.42,1.53,8.09
­,24.00,4.33,2.32,11.33
skriver,21.00,4.38,2.29,11.35
oss,21.00,4.57,2.20,11.45
opplyser,22.00,4.50,2.28,11.63
dem,26.00,4.54,2.27,11.65
sier,137.00,4.33,2.50,12.11
ukene,23.00,4.61,2.35,12.21
disse,42.00,4.62,2.41,12.51
De,46.00,4.33,2.64,12.73


In [571]:
cutoff = 20
(right_df[(right_df['count'] >= cutoff)].sort_values(by=['score'])
 .head(20).dropna()
 .style.format(precision=2)
  .set_table_styles([{'selector':"th", 'props':'font-size:18px'},
                   {'selector':'td', 'props':'font-size:18px'},
                   {'selector':'tc', 'props':'font-size:18px'}])
)

,count,mean,std,score
Pfizer-vaksinen,22.00,1.64,1.26,2.55
koronavaksine,51.00,1.29,1.74,2.63
per,44.00,1.09,2.23,2.76
beskyttelse,43.00,2.86,1.66,5.59
■,24.00,1.67,3.07,5.62
gir,39.00,1.82,2.86,5.74
glass,41.00,2.83,1.73,5.75
vaksine,104.00,2.22,2.37,5.94
enn,122.00,1.95,2.78,6.00
uka,47.00,2.28,2.40,6.15


In [ ]:
(right_df[(right_df['count'] >= cutoff)].sort_values(by=['score'])
 .head(20)

In [570]:
(left_df[(left_df['count'] >= cutoff) & (left_df['mean']>4.3) &
         (left_df['mean'] < 4.7)]
 .sort_values(by=['score']).dropna()
 .head(10)
 .style.format(precision=2)
 .set_table_styles([{'selector':"th", 'props':'font-size:18px'},
                   {'selector':'td', 'props':'font-size:18px'},
                   {'selector':'tc', 'props':'font-size:18px'}])
)

,count,mean,std,score
mulig,21.00,4.57,1.75,9.36
man,61.00,4.51,1.87,9.77
når,42.00,4.52,1.92,10.02
hetteglass,24.00,4.42,2.26,11.32
skulle,39.00,4.56,2.22,11.51
vi,274.00,4.40,2.37,11.75
kan,92.00,4.64,2.24,11.80
hadde,42.00,4.69,2.23,11.84
:,126.00,4.55,2.38,12.20
ble,63.00,4.40,2.49,12.25


In [575]:
left_df['mean'].describe()

count    733.000000
mean       4.226154
std        1.840448
min        0.000000
25%        2.888889
50%        4.600000
75%        5.592593
max        8.200000
Name: mean, dtype: float64

In [577]:
right_df['mean'].describe()

count    664.000000
mean       4.496259
std        1.372292
min        0.000000
25%        3.698913
50%        4.572381
75%        5.419643
max        8.583333
Name: mean, dtype: float64

In [576]:
left_df['std'].describe()

count    733.000000
mean       2.175524
std        0.795994
min        0.000000
25%        1.765949
50%        2.265686
75%        2.691277
max        4.230839
Name: std, dtype: float64

In [578]:
right_df['std'].describe()

count    664.000000
mean       2.429594
std        0.658147
min        0.000000
25%        2.073644
50%        2.514366
75%        2.851584
max        4.381780
Name: std, dtype: float64

In [573]:
(left_df[(left_df['count'] >= cutoff)]
 .sort_values(by=['score']).dropna()
 .head(20)
 .style.format(precision=2)
 .set_table_styles([{'selector':"th", 'props':'font-size:18px'},
                   {'selector':'td', 'props':'font-size:18px'},
                   {'selector':'tc', 'props':'font-size:18px'}])
)

,count,mean,std,score
milliarder,39.00,0.28,1.30,0.45
200.000,21.00,0.33,1.11,0.47
begge,21.00,0.57,1.25,0.88
færre,72.00,0.47,1.62,0.91
million,32.00,0.75,1.55,1.38
Du,23.00,2.09,0.42,1.50
mange,206.00,0.73,2.05,1.72
store,59.00,0.75,2.06,1.76
to,499.00,0.77,2.08,1.82
trenger,32.00,1.47,1.39,2.48


In [493]:
plot_series(pd.Series(left_words['diktatur']))

alt.Chart(...)

In [494]:
plot_series(pd.Series(right_words['diktatur']))

alt.Chart(...)

In [495]:
plot_series(pd.Series(left_words['eller']))

alt.Chart(...)

In [ ]:
plot_series(pd.Series(left_words['eller']))

In [360]:
plot_series(pd.Series(left_words['små']))

alt.Chart(...)

In [361]:
plot_series(pd.Series(left_words['flere']))

alt.Chart(...)

In [502]:
(left_df
 .loc['diktatur aristokrati demokrati .'.split()]
 .sort_values(by='score')
 .style
 .set_table_styles([{'selector':"th", 'props':'font-size:18px'},
                   {'selector':'td', 'props':'font-size:18px'},
                   {'selector':'tc', 'props':'font-size:18px'}])
 .format(precision=2)
)

,count,mean,std,score
aristokrati,95.00,2.33,2.01,5.38
diktatur,258.00,2.75,2.41,7.44
demokrati,509.00,5.72,2.35,15.17
.,6282.00,5.69,2.41,15.40


# Right context for "doser"

In [503]:
(right_df
 .loc['aristokrati diktatur demokrati .'.split()]
 .sort_values(by='mean')
 .style
 .format(precision=2)
 .set_table_styles([{'selector':"th", 'props':'font-size:18px'},
                   {'selector':'td', 'props':'font-size:18px'},
                   {'selector':'tc', 'props':'font-size:18px'}])
 .format(precision=2)
)

,count,mean,std,score
aristokrati,47.00,2.36,2.28,6.09
.,13964.00,2.94,2.98,9.65
diktatur,313.00,2.97,2.39,8.01
demokrati,509.00,5.53,2.37,14.78


In [222]:
df.loc[['Pfizer','eller','vaksinert','FHI','antall','store','mange']].sort_values(by='mean').style.format(precision=2)

,count,mean,std
store,58.00,0.71,2.06
mange,188.00,0.80,2.13
antall,88.00,0.98,2.51
eller,54.00,3.28,2.66
vaksinert,59.00,3.98,2.38
FHI,55.00,4.69,2.47
Pfizer,39.00,5.49,2.56


In [504]:
c = dh.Concordance(corpus, query="NEAR(kulturelt demokrati)")

In [505]:
c.size

82

In [507]:
c.show(50)

,link,concordance
60,URN:NBN:no-nb_pliktmonografi_000001231,... av Stat Fylke / kommune Næringsliv Forankring Nasjonen Lokalsamfunnet Verden Motto Demokratisering av kultur Kulturelt demokrati Utbytte / lønnsomhet Rasjonalitet...
68,URN:NBN:no-nb_digibok_2020042248015,"Balterne gikk ut i gatene og krevde kulturelt og politisk selvstyre , demokrati og rettferdighet . Deres kampsaker var således..."
58,URN:NBN:no-nb_digibok_2022101448081,"... overfor Frihed og politisk , retsligt , kulturelt , socialt og økonomisk Demokrati tænker sig , at de tilsvarende Love..."
64,URN:NBN:no-nb_pliktmonografi_000013079,"... I stedet var det « kulturelt demokrati » som gjaldt , og da i betydningen at det var perspektiver , interesser og..."
30,URN:NBN:no-nb_digibok_2009072101019,KUNSTFORMIDLING PÅ 90 - TALLET - KULTURELT DEMOKRATI OG KUNSTFELTETS AUTONOMI
35,URN:NBN:no-nb_digibok_2014120807552,"... Her settes målene klarere , også tankene om kulturelt demokrati , « kultur for alle » , med egenaktivitet som et gode..."
59,URN:NBN:no-nb_pliktmonografi_000001231,"... Demokratisering av kulturen , som betegner den gamle « kulturspreiingstanken » , og kulturelt demokrati som åpnet for et mer pluralistisk verdigrunnlag..."
19,URN:NBN:no-nb_digibok_2009072101019,5 KUNSTFORMIDLING PÅ 90 - TALLET - KULTURELT DEMOKRATI OG KUNSTFELTETS AUTONOMI 91
34,URN:NBN:no-nb_digibok_2009072101019,... Begge prosjektene holder fast ved en forståelse av kunstformidling som knytter an til tanker om kulturelt demokrati slik de kom...
66,URN:NBN:no-nb_pliktmonografi_000013475,"... menneskeverd , identitet og kulturelt mangfold , kritisk tenkning og etisk bevissthet , og demokrati og medvirkning . Til slutt..."


In [391]:
g = dp.Graph()
states = ["doser", "med", "vaksine"]
for state in states:
    g.setNode(state, tooltip='tooltip1 of ' + state)
g.setEdge("doser",     "med",     label="2.9")
g.setEdge("med",     "vaksine",   label="3.0")
g.setEdge("doser",     "vaksine",   label="2.5")


In [392]:
widget = dp.DagreD3Widget(graph=g)
widget

DagreD3Widget()

In [395]:
right_df['score'] = right_df['mean']*right_df['std']

In [398]:
right_df.sort_values(by = 'score').head(20)

,count,mean,std,score
narkotika,12.0,0.000000,0.000000,0.000000
pr.,12.0,0.000000,0.000000,0.000000
varme,5.0,0.000000,0.000000,0.000000
russiske,5.0,2.000000,0.000000,0.000000
ulovlig,5.0,7.000000,0.000000,0.000000
yUke,5.0,0.400000,0.547723,0.219089
kokain,7.0,0.428571,0.534522,0.229081
leveres,6.0,1.166667,0.408248,0.476290
innbygger,9.0,1.555556,0.527046,0.819850
Astra-,6.0,1.000000,1.095445,1.095445
